ORGANIZACIJA:
evo linka od prezentacije:
https://fb.watch/4y-LKdA7iT/

UPUTE:
pokrenuti zakomentiranu skriptu ispod i uploadat si pkl file na google disk.

In [174]:
# import pickle

# start_time = time.time()
# tcurrent = start_time
# np.random.seed(33)

# filename = 'LUMEN_DS.csv'
# df = pd.read_csv(filename, encoding="UTF-16LE", sep='|', quotechar='"', low_memory=False)

# to_drop_too_much_missing = ['Price last modified date in the ERP',
#                             'Sales Channel - Grouping',
#                             'Material cost of part',
#                             'Labor cost of part',
#                             'Overhead cost of part']

# df.drop(to_drop_too_much_missing, inplace=True, axis=1)

# to_drop_unimportant = [
#     'Sales Channel - Internal',
#     'Sales Channel - External']

# df.drop(to_drop_unimportant, inplace=True, axis=1)

# with open('LUMEN2.pkl', 'wb') as f:
#     pickle.dump(df, f)

In [175]:
!pip install sweetviz

In [176]:
import pickle
import matplotlib.pyplot as plt
from google.colab import drive
import os
import time
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
import sweetviz as sv
import xgboost as xgb
import pickle
from sklearn import metrics
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [177]:
data_path = "/content/drive/MyDrive/LUMEN0.pkl"
with open(data_path, 'rb') as f:
    df = pickle.load(f)
print(df.shape)

(1294962, 33)


Outlieri i dalje nisu gotovi, automatska detekcija pomocu IsolationForest ili LocalOutlierFactor mi nije davala dobre rezultate.

Zasad sam samo s desne strane krivulje rucno odbacio vrijednosti vece od q3+1.5iqr. 

Kako upariti 

In [178]:
df = df[~df['Invoiced price'].isnull()]

df = df[~df['GM%'].isnull()]
df = df[df['GM%'] >= -1]  # ???
df = df[df['GM%'] <= 1]  # ???
df = df[df['Invoiced price'] > 0]
df = df[df['Cost of part'] >= 0]
df = df[~df['GM%'].isnull()]
df = df[~df['Item Code'].isnull()]

#0 je po meni legalna vrijednost za invoiced quantity, ali ne i za ordered
df = df[df['Invoiced qty (shipped)'] >= 0]
df = df[df['Invoiced qty (shipped)'] < 1000000]
df = df[df['Ordered qty'] > 0]
df = df[df['Ordered qty'] < 1000000]
df = df[df['Invoiced price'] > 0]
df = df[df['Invoiced price'] < 100000]
df = df[df['Cost of part'] >= 0]
df = df[df['Cost of part'] < 75000]

df = df[~df['# of unique products on a quote'].isnull()]
df = df[df['# of unique products on a quote'] < 374]
df = df[df['# of unique products on a quote'] > 0]

#simple drop
df = df[~df['Product group'].isnull()]
df = df[~df['Manufacturing Location Code'].isnull()]
df = df[~df['Make vs Buy'].isnull()]
df = df[~df['Customer industry'].isnull()]

#ovo nije dobro, maknuo sam ih jer ih je malo. Jel se ovdje moze koristiti SMOTE?
# df = df[df['Product family']!='PC010']
# df = df[df['Product family']!='PC001']
# df = df[df['Product family']!='PC016']

# df[['Material cost of part',
#     'Labor cost of part',
#     'Overhead cost of part'
#     ]] = df[['Material cost of part',
#              'Labor cost of part',
#              'Overhead cost of part'
#              ]].fillna(value=0)
# df['sum'] = df['Material cost of part'] + df['Labor cost of part'] + df['Overhead cost of part']
# df['sum'] = np.where(df['sum']==0, df['Cost of part'], df['sum'])

df['Invoiced qty (shipped)'].astype(int)
df['Ordered qty'].astype(int)
df['# of unique products on a quote'].astype(int)

# scaler = MinMaxScaler()
# df[['GM%']] = scaler.fit_transform(df[['GM%']])

df['Invoiced qty (shipped)'] = np.log10(df['Invoiced qty (shipped)'] + 1)
df['Ordered qty'] = np.log10(df['Ordered qty'] + 1)
df['Cost of part'] = np.log10(df['Cost of part'] + 1)
df['Invoiced qty (shipped)2'] = np.log10(df['Invoiced qty (shipped)'] + 1)*2
df['Ordered qty2'] = np.log10(df['Ordered qty'] + 1)*2
df['Cost of part2'] = np.log10(df['Cost of part'] + 1)*2
df['Invoiced qty (shipped)3'] = np.log10(df['Invoiced qty (shipped)'] + 1)*3
df['Ordered qty3'] = np.log10(df['Ordered qty'] + 1)*3
df['Cost of part3'] = np.log10(df['Cost of part'] + 1)*3
df['Invoiced price'] = np.log10(df['Invoiced price'] + 1)#????????????????????????????????????????????????
df['# of unique products on a quote'] = np.log10(df['# of unique products on a quote'] + 1)

In [179]:
to_drop_too_much_missing = ['Price last modified date in the ERP',
                            'Sales Channel - Grouping',
                            'Material cost of part',
                            'Labor cost of part',
                            'Overhead cost of part']

df.drop(to_drop_too_much_missing, inplace=True, axis=1)

to_drop_unimportant = [
    'Sales Channel - Internal',
    'Sales Channel - External']

df.drop(to_drop_unimportant, inplace=True, axis=1)

Na datumima jos uvijek radim, glavna ideja mi je za feature uzeti broj dana izmedju 'Customer First Invoice Date',
                 'Born on date',
                 'Order Date' 
i  Invoice Date-a

In [180]:
to_drop_highly_correlated = [#'Invoiced price',
                             'Invoiced price (TX)']

df.drop(to_drop_highly_correlated, inplace=True, axis=1)

In [181]:
to_drop_dates = ['Customer First Invoice Date',
                 'Born on date',
                 'Invoice Date',
                 'Order Date']

df.drop(to_drop_dates, inplace=True, axis=1)

Prvih 6 featura u listi su svojevrsni id-evi, koristit ce kod bilo kakvog grupiranja i punjenja praznih vrijednosti(na tom radim).

Zadnja 4 su kategoricki koje jos nisam uklopio. Postoji hashing trick, ali sumnjam obzirom na mali broj klasa da je to najbolje rjesenje.

In [182]:
to_drop_need_fe = [
                   'Invoice Line #',#ima date u sebi?
                   'Order Line #',
                   'Invoice #',
                   'Order #',
                   '# of unique products on a quote'#droppam svejedno jer je losa korelacija
                   ]

df.drop(to_drop_need_fe, inplace=True, axis=1)

Ideje za neke nove feature

iz worda:

Grupirati po transakcijama

Stvari koje se narucuju u grupama - prihvatljiviji manji GM%

Relationship between order, invoice and deal

Usporedba razlicitih salesamana

Clustering customers

Kako se mjenja cjena itema kroz vrijeme


In [183]:
#df['itemqty0'] = df.groupby('Order #')['Item Code'].transform('nunique')
df['itemqty_fb'] = df.groupby(['CustomerID', 'Customer industry', 'Manufacturing Region', 'Product group'])[
    'Item Code'].transform('nunique')
df['total_part_cost']=df['Invoiced qty (shipped)']*df['Cost of part']
df['deal_size'] = df.groupby(['CustomerID', 'Customer industry', 'Manufacturing Region', 'Product group'])[
    'total_part_cost'].transform('sum')
df['deal_qty'] = df.groupby(['CustomerID', 'Customer industry', 'Manufacturing Region', 'Product group'])[
    'Invoiced qty (shipped)'].transform('sum')

#df = df[df['itemqty_fb'] < 500]

df['itemqty_fb'] = np.log10(df['itemqty_fb'] + 1)
df['deal_size'] = np.log10(df['deal_size'] + 1)
df['deal_qty'] = np.log10(df['deal_qty'] + 1)
df['itemqty_fb2'] = np.log10(df['itemqty_fb'] + 1)*2
df['deal_size2'] = np.log10(df['deal_size'] + 1)*2
df['deal_qty2'] = np.log10(df['deal_qty'] + 1)*2
df['itemqty_fb3'] = np.log10(df['itemqty_fb'] + 1)*3
df['deal_size3'] = np.log10(df['deal_size'] + 1)*3
df['deal_qty3'] = np.log10(df['deal_qty'] + 1)*3

In [184]:
to_drop_need_fe2 = ['CustomerID',
                   'Item Code',
                  #  'Product group',
                  #  'Customer industry'
                  #   'Manufacturing Location Code',
                  #  'Make vs Buy',
                   ]

df.drop(to_drop_need_fe2, inplace=True, axis=1)

In [185]:
# dummies = ['Manufacturing Region',
#            'Intercompany',
#            'Customer Region',
#            'Product family',
#            'Top Customer Group',
            #'Product group',
            #'Manufacturing Location Code'
            # 'Make vs Buy',
            # 'Customer industry'


# df.drop(dummies, inplace=True, axis=1)
df = pd.get_dummies(df, drop_first=True)

Mozda ce trebat dvoklik na mapu Files gore lijevo da se refresha, downloadajte html i otvorite s browserom.

In [186]:
# sv_report = sv.analyze(df, target_feat='Invoiced price')
# sv_report.show_html('SV_report_lumen20.html')

Jel vrijedi tvrdnja:
Sto je dobro za linearnu regresiju dobro je za sve modele?
Tj. moze li nam ovo posluzit za evaluaciju u feature engineeringu prije nego uopce krenemo razmisljati o modelu?

In [187]:
# regr2 = DecisionTreeRegressor(max_depth=10)
# regr2.fit(x_train, y_train)
# y_trpred=regr2.predict(x_train)
# y_pred=regr2.predict(x_valid)

In [188]:
#df['miss_gm'] = (df['Invoiced price'] - df['Cost of part']) / df['Invoiced price']

In [189]:
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(df.drop('Invoiced price', axis=1), df['Invoiced price'], test_size=0.25)


regr2 = LinearRegression()
regr2.fit(x_train.drop('GM%', axis=1), y_train)
y_trpred=regr2.predict(x_train.drop('GM%', axis=1))
y_pred=regr2.predict(x_valid.drop('GM%', axis=1))

In [190]:
to_drop_output = ['Invoiced price']

df.drop(to_drop_output, inplace=True, axis=1)

In [191]:
y_train=x_train['GM%']
y_valid=x_valid['GM%']

miss_gm_tr = (y_trpred - x_train['Cost of part']) / y_trpred
#miss_gm_tr.replace([np.inf, -np.inf], 1.0, inplace=True)

rmse_tr=np.sqrt(metrics.mean_squared_error(miss_gm_tr,y_train))
print(rmse_tr)
r2_tr=regr2.score(x_train.drop('GM%', axis=1), miss_gm_tr)
print(r2_tr)

miss_gm = (y_pred - x_valid['Cost of part']) / y_pred
#miss_gm.replace([np.inf, -np.inf], 1.0, inplace=True)

print()
rmse=np.sqrt(metrics.mean_squared_error(miss_gm,y_valid))
print(rmse)
r2=regr2.score(x_valid.drop('GM%', axis=1), miss_gm)
print(r2)

38.801420444227745
-0.0006456771461378352

51.328798202097886
-0.0004429799065766016
